# Implementacja modelu uczenia maszynowego na potrzeby predykcji kursu indeksu WIG20

Ideą numer 1 jest próba wyłonienia najbardziej istotnych informacji, które zwiększają prawdopodobieństwo podjęcia właściwej decyzji inwestycyjnej.
Ideą numer 2 jest stworzenie modelu uczenia maszynowego, aby sprawdzić jego przydatność w tradingu. Używana do budowy modeli jest bibliteka scikit-learn.
Idea numer 3 to próba wyłonienia powiązań między wskaźnikami, które z wyprzedzeniem informują o zbliżających się sygnałach inwestycyjnych

### Instalacja niezbędnych bibliotek

Bazuję na danych historycznych pochodzących z serwisu: https://www.dukascopy.com/swiss/pl/marketwatch/historical/
Dukascopy daje możliwość pobierania danych za pomocą specjalnej biblioteki pythonowej, co jest bardzo wygodnym rozwiązaniem. Nazwa biblioteki: dukascopy-python

In [114]:
#pip install dukascopy-python

In [115]:
#pip install --upgrade dukascopy-python

Biblioteka dukascopy-python pobiera dane prosto do struktury danych Pandas DataFrame, więc trzeba zainstalować biblitekę Pandas

In [116]:
#pip install pandas

In [117]:
#pip install --upgrade pandas

Sprawdzenie możliwości biblioteki

Niezbędna będzie malipulacja datami: biblioteka datetime

In [118]:
#pip install datetime

In [119]:
#pip install --upgrade datetime

Skoro ML to biblioteka scikit-learn

In [120]:
#pip install scikit-learn

In [121]:
#pip intall --upgrade scikit-learn

Jakaś biblioteka do analizy technicznej

In [122]:
#pip install pandas-ta

In [123]:
#pip install --ipgrade pandas-ta

### Import bibliotek

In [124]:
from datetime import datetime
#
import dukascopy_python as dcp
#
import pandas as pd
#
import pandas_ta as ta

Wyświetlenie dokumentacji biblioteki dukascopy_python

In [125]:
help(dcp)

Help on package dukascopy_python:

NAME
    dukascopy_python

PACKAGE CONTENTS
    instruments

FUNCTIONS
    fetch(instrument: str, interval: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    live_fetch(instrument: str, interval_value: int, time_unit: str, offer_side: str, start: datetime.datetime, end: datetime.datetime, max_retries: int = 7, limit: int = 30000, debug=False)

    sleep(...)
        sleep(seconds)

        Delay execution for a given number of seconds.  The argument may be
        a floating-point number for subsecond precision.

DATA
    INTERVAL_DAY_1 = '1DAY'
    INTERVAL_HOUR_1 = '1HOUR'
    INTERVAL_HOUR_4 = '4HOUR'
    INTERVAL_MIN_1 = '1MIN'
    INTERVAL_MIN_10 = '10MIN'
    INTERVAL_MIN_15 = '15MIN'
    INTERVAL_MIN_30 = '30MIN'
    INTERVAL_MIN_5 = '5MIN'
    INTERVAL_MONTH_1 = '1MONTH'
    INTERVAL_SEC_1 = '1SEC'
    INTERVAL_SEC_10 = '10SEC'
    INTERVAL_SEC_30 = '30SEC'
    I

Sprawdzenie listy dostępnych instrumentów

In [126]:
import dukascopy_python.instruments as dcpi

In [127]:
help(dcpi)

Help on module dukascopy_python.instruments in dukascopy_python:

NAME
    dukascopy_python.instruments

DATA
    INSTRUMENT_AUSTRIA_ANDR_AT_EUR = 'ANDR.AT/EUR'
    INSTRUMENT_AUSTRIA_EBS_AT_EUR = 'EBS.AT/EUR'
    INSTRUMENT_AUSTRIA_POST_AT_EUR = 'POST.AT/EUR'
    INSTRUMENT_AUSTRIA_RBI_AT_EUR = 'RBI.AT/EUR'
    INSTRUMENT_AUSTRIA_TKA_AT_EUR = 'TKA.AT/EUR'
    INSTRUMENT_AUSTRIA_VER_AT_EUR = 'VER.AT/EUR'
    INSTRUMENT_AUSTRIA_VIG_AT_EUR = 'VIG.AT/EUR'
    INSTRUMENT_AUSTRIA_VOE_AT_EUR = 'VOE.AT/EUR'
    INSTRUMENT_AUSTRIA_WIE_AT_EUR = 'WIE.AT/EUR'
    INSTRUMENT_BELGIUM_ABI_BE_EUR = 'ABI.BE/EUR'
    INSTRUMENT_BELGIUM_AGS_BE_EUR = 'AGS.BE/EUR'
    INSTRUMENT_BELGIUM_BELG_BE_EUR = 'BELG.BE/EUR'
    INSTRUMENT_BELGIUM_KBC_BE_EUR = 'KBC.BE/EUR'
    INSTRUMENT_BELGIUM_SOLB_BE_EUR = 'SOLB.BE/EUR'
    INSTRUMENT_BELGIUM_UCB_BE_EUR = 'UCB.BE/EUR'
    INSTRUMENT_BELGIUM_UMI_BE_EUR = 'UMI.BE/EUR'
    INSTRUMENT_BND_CFD_BUND_TR_EUR = 'BUND.TR/EUR'
    INSTRUMENT_BND_CFD_UKGILT_TR_GBP = 'UKGILT.

Sprawdzenie możliwości biblioteki Pandas TA

In [128]:
help(ta)

Help on package pandas_ta:

NAME
    pandas_ta

PACKAGE CONTENTS
    candles (package)
    core
    custom
    cycles (package)
    momentum (package)
    overlap (package)
    performance (package)
    statistics (package)
    trend (package)
    utils (package)
    volatility (package)
    volume (package)

SUBMODULES
    alphavantage
    data
    yahoofinance

FUNCTIONS
    localtime(...)
        localtime([seconds]) -> (tm_year,tm_mon,tm_mday,tm_hour,tm_min,
                                  tm_sec,tm_wday,tm_yday,tm_isdst)

        Convert seconds since the Epoch to a time tuple expressing local time.
        When 'seconds' is not passed in, convert the current time instead.

    mfloor = floor(x, /)
        Return the floor of x as an Integral.

        This is the largest integer <= x.

    mul(a, b, /)
        Same as a * b.

    npArray = array(...)
        array(object, dtype=None, *, copy=True, order='K', subok=False, ndmin=0,
              like=None)

        Create an arra

Sprawdzenie jakie wskaźniki i funkcje są dostępne w bibliotece Pandas-TA

In [129]:
df = pd.DataFrame()
lista_wskaźników = [metoda for metoda in dir(df.ta) if not metoda.startswith('_')]

print(f"Znaleziono {len(lista_wskaźników)} wskaźników i funkcji.")
print(lista_wskaźników)

Znaleziono 157 wskaźników i funkcji.
['aberration', 'above', 'above_value', 'accbands', 'ad', 'adjusted', 'adosc', 'adx', 'alma', 'amat', 'ao', 'aobv', 'apo', 'aroon', 'atr', 'bbands', 'below', 'below_value', 'bias', 'bop', 'brar', 'categories', 'cci', 'cdl_pattern', 'cdl_z', 'cfo', 'cg', 'chop', 'cksp', 'cmf', 'cmo', 'constants', 'coppock', 'cores', 'cross', 'cross_value', 'cti', 'datetime_ordered', 'decay', 'decreasing', 'dema', 'dm', 'donchian', 'dpo', 'ebsw', 'efi', 'ema', 'entropy', 'eom', 'er', 'eri', 'exchange', 'fisher', 'fwma', 'ha', 'hilo', 'hl2', 'hlc3', 'hma', 'hwc', 'hwma', 'ichimoku', 'increasing', 'indicators', 'inertia', 'jma', 'kama', 'kc', 'kdj', 'kst', 'kurtosis', 'kvo', 'last_run', 'linreg', 'log_return', 'long_run', 'macd', 'mad', 'massi', 'mcgd', 'median', 'mfi', 'midpoint', 'midprice', 'mom', 'natr', 'nvi', 'obv', 'ohlc4', 'pdist', 'percent_return', 'pgo', 'ppo', 'psar', 'psl', 'pvi', 'pvo', 'pvol', 'pvr', 'pvt', 'pwma', 'qqe', 'qstick', 'quantile', 'reverse', 'r

### Definicja paramertów dla dla funkcji fetch() z biblioteki dukascopy_python

In [130]:
instrument = 'EUR/USD'
interval = '1DAY'
offer_side = 'B' 
start_date = datetime(2000, 1, 1)
end_date = datetime.today()

### Pobranie danych do Pandas DataFrame

In [131]:
df = dcp.fetch(instrument, interval, offer_side, start_date, end_date)

INFO:DUKASCRIPT:current timestamp :2025-08-01T02:00:00


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    7152 non-null   float64
 1   high    7152 non-null   float64
 2   low     7152 non-null   float64
 3   close   7152 non-null   float64
 4   volume  7152 non-null   float64
dtypes: float64(5)
memory usage: 335.2 KB


Zbiór jest kompletny, zgodnie z oczekiwaniami zawiera dane do dnia poprzedzającego dzień uruchomienia skryptu.
Pora zatem na rozbudowanie zbioru i policzenie interesujących nas statystyk, wskaźników i miar.

Wskaźniki trendu

1. adx (Average Directional Index)
2. amat (Archer Moving Average Trend)
3. aroon (Aroon Indicator)
4. cti (Correlation Trend Indicator)
5. dm (Directional Movement)
6. ebsw (Even Better Sine Wave)
7. er (Efficiency Ratio)
8. ichimoku (Ichimoku Kinko Hyo)
9. linreg (Linear Regression)
10. psar (Parabolic Stop and Reverse)
11. slope (Slope)
12. supertrend (Supertrend)
13. td_seq (TD Sequential)
14. ttm_trend (TTM Trend)
15. vhf (Vertical Horizontal Filter)
16. vortex (Vortex Indicator)

In [133]:
# Average Directional Index, ustawienia standardowe
# Default Inputs: length=14, lensig=lenght, scalar=100, mamode='rma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,22.819567,20.076440,27.971068
2025-07-29 00:00:00+00:00,23.094972,18.568266,32.078387
2025-07-30 00:00:00+00:00,24.348391,15.824123,37.494199


In [134]:
# Archer Moving Average Trend, ustawienia standardowe
# Default Inputs: fast=8, slow=21, lookback=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.amat(append=True)

,AMATe_LR_8_21_2,AMATe_SR_8_21_2
timestamp,,
2003-05-04 00:00:00+00:00,0,0
2003-05-05 00:00:00+00:00,0,0
2003-05-06 00:00:00+00:00,0,0
2003-05-07 00:00:00+00:00,0,0
2003-05-08 00:00:00+00:00,0,0
...,...,...
2025-07-28 00:00:00+00:00,0,1
2025-07-29 00:00:00+00:00,0,1
2025-07-30 00:00:00+00:00,0,1


In [135]:
# Aroon Indicator, ustawienia standardowe
# Default Inputs: length=14, scalar=100, talib=True, offset=0 <- parametry do modyfikacji
df.ta.aroon(append=True)

,AROOND_14,AROONU_14,AROONOSC_14
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,35.714286,78.571429,42.857143
2025-07-29 00:00:00+00:00,100.000000,71.428571,-28.571429
2025-07-30 00:00:00+00:00,100.000000,64.285714,-35.714286


In [136]:
# Correlation Trend Indicator, ustawienia standardowe
# Default Inputs: length=12, offset=0 <- parametry do modyfikacji
df.ta.cti(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.547996
2025-07-29 00:00:00+00:00    0.101664
2025-07-30 00:00:00+00:00   -0.319069
2025-07-31 00:00:00+00:00   -0.587382
2025-08-01 00:00:00+00:00   -0.675145
Name: CTI_12, Length: 7152, dtype: float64

In [137]:
# Directional Movement, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.dm(append=True)

,DMP_14,DMN_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.001523,0.002122
2025-07-29 00:00:00+00:00,0.001414,0.002443
2025-07-30 00:00:00+00:00,0.001313,0.003112


In [138]:
# Even Better Sine Wave, ustawienia standardowe
# Default Inputs: length=40, bars=10, offset=0 <- parametry do modyfikacji
df.ta.ebsw(append=True)

C:\Users\mt242\AppData\Local\Temp\ipykernel_43464\965367681.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.ta.ebsw(append=True)


timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.156943
2025-07-29 00:00:00+00:00   -0.612977
2025-07-30 00:00:00+00:00   -0.918954
2025-07-31 00:00:00+00:00   -0.984813
2025-08-01 00:00:00+00:00   -0.939068
Name: EBSW_40_10, Length: 7152, dtype: float64

In [139]:
# Efficiency Ratio, ustawienia standardowe
# Default Inputs: length=10, drift=1, offset=0 <- parametry do modyfikacji
df.ta.er(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.100723
2025-07-29 00:00:00+00:00    0.156083
2025-07-30 00:00:00+00:00    0.364168
2025-07-31 00:00:00+00:00    0.394043
2025-08-01 00:00:00+00:00    0.178887
Name: ER_10, Length: 7152, dtype: float64

In [140]:
# Ichimoku Kinko Hyo, ustawienia standardowe
# Default Inputs: tenkan=9, kijun=26, senkou=52, include_chikou=True, offset=0 <- parametry do modyfikacji
df.ta.ichimoku(append=True)

(                              ISA_9    ISB_26     ITS_9    IKS_26   ICS_26
 timestamp                                                                 
 2003-05-04 00:00:00+00:00       NaN       NaN       NaN       NaN  1.17337
 2003-05-05 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16444
 2003-05-06 00:00:00+00:00       NaN       NaN       NaN       NaN  1.18397
 2003-05-07 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16973
 2003-05-08 00:00:00+00:00       NaN       NaN       NaN       NaN  1.16989
 ...                             ...       ...       ...       ...      ...
 2025-07-28 00:00:00+00:00  1.154422  1.140950  1.168685  1.169320      NaN
 2025-07-29 00:00:00+00:00  1.156655  1.140950  1.165375  1.167430      NaN
 2025-07-30 00:00:00+00:00  1.159455  1.142790  1.159475  1.161530      NaN
 2025-07-31 00:00:00+00:00  1.161633  1.144745  1.159475  1.160540      NaN
 2025-08-01 00:00:00+00:00  1.161742  1.144745  1.159010  1.160075      NaN
 
 [7152 row

In [141]:
# Linear Regression, ustawienia standardowe
# Default Inputs: length=10, offset=0 <- parametry do modyfikacji
df.ta.linreg(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    1.171663
2025-07-29 00:00:00+00:00    1.168764
2025-07-30 00:00:00+00:00    1.163186
2025-07-31 00:00:00+00:00    1.157406
2025-08-01 00:00:00+00:00    1.156071
Name: LR_14, Length: 7152, dtype: float64

In [142]:
# Parabolic Stop and Reverse, ustawienia standardowe
# Default Inputs: af0=0.02, af=0.02, max_af=0.2, offset=0 <- parametry do modyfikacji
df.ta.psar(append=True)

,PSARl_0.02_0.2,PSARs_0.02_0.2,PSARaf_0.02_0.2,PSARr_0.02_0.2
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,0.02,0
2003-05-05 00:00:00+00:00,NaN,1.130090,0.02,1
2003-05-06 00:00:00+00:00,1.120140,NaN,0.02,1
2003-05-07 00:00:00+00:00,1.120140,NaN,0.02,0
2003-05-08 00:00:00+00:00,1.120638,NaN,0.04,0
...,...,...,...,...
2025-07-28 00:00:00+00:00,NaN,1.178870,0.02,1
2025-07-29 00:00:00+00:00,NaN,1.178463,0.04,0
2025-07-30 00:00:00+00:00,NaN,1.177399,0.06,0


In [143]:
# Slope, ustawienia standardowe
# Default Inputs: length=1, as_angle=False, to_degrees=False, vertical=None, offset=0 <- parametry do modyfikacji
df.ta.slope(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00755
2003-05-06 00:00:00+00:00    0.01310
2003-05-07 00:00:00+00:00   -0.00740
2003-05-08 00:00:00+00:00    0.01326
                              ...   
2025-07-28 00:00:00+00:00   -0.01591
2025-07-29 00:00:00+00:00   -0.00457
2025-07-30 00:00:00+00:00   -0.01228
2025-07-31 00:00:00+00:00   -0.00061
2025-08-01 00:00:00+00:00    0.01613
Name: SLOPE_1, Length: 7152, dtype: float64

In [144]:
# Supertrend, ustawienia standardowe
# Default Inputs: length=7, multiplier=3.0, offset=0 <- parametry do modyfikacji
df.ta.supertrend(append=True)

,SUPERT_7_3.0,SUPERTd_7_3.0,SUPERTl_7_3.0,SUPERTs_7_3.0
timestamp,,,,
2003-05-04 00:00:00+00:00,0.000000,1,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,1,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,1,NaN,NaN
...,...,...,...,...
2025-07-28 00:00:00+00:00,1.159619,1,1.159619,NaN
2025-07-29 00:00:00+00:00,1.179974,-1,NaN,1.179974
2025-07-30 00:00:00+00:00,1.176671,-1,NaN,1.176671


In [145]:
# TD Sequential, ustawienia standardowe
# Default Inputs: asint=False, offset=0 <- parametry do modyfikacji
df.ta.td_seq(append=True)

,TD_SEQ_UP,TD_SEQ_DN
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
7147,NaN,NaN
7148,NaN,NaN
7149,NaN,NaN
7150,NaN,NaN


In [146]:
# TTM Trend, ustawienia standardowe
# Default Inputs: length=6, offset=0 <- parametry do modyfikacji
df.ta.ttm_trend(append=True)

,TTM_TRND_6
timestamp,
2003-05-04 00:00:00+00:00,-1
2003-05-05 00:00:00+00:00,-1
2003-05-06 00:00:00+00:00,-1
2003-05-07 00:00:00+00:00,-1
2003-05-08 00:00:00+00:00,-1
...,...
2025-07-28 00:00:00+00:00,-1
2025-07-29 00:00:00+00:00,-1
2025-07-30 00:00:00+00:00,-1


In [147]:
# Vertical Horizontal Filter, ustawienia standardowe
# Default Inputs: length=28, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vhf(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.265252
2025-07-29 00:00:00+00:00    0.309263
2025-07-30 00:00:00+00:00    0.409022
2025-07-31 00:00:00+00:00    0.417447
2025-08-01 00:00:00+00:00    0.375969
Name: VHF_28, Length: 7152, dtype: float64

In [148]:
# Vortex Indicator, ustawienia standardowe
# Default Inputs: length=14, drift=1, offset=0 <- parametry do modyfikacji
df.ta.vortex(append=True)

,VTXP_14,VTXM_14
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,0.912482,0.934432
2025-07-29 00:00:00+00:00,0.861477,1.065235
2025-07-30 00:00:00+00:00,0.786592,1.090566


Oscylatory i wskaźniki pędu (momentum)

1. ao (Awesome Oscillator)
2. apo (Absolute Price Oscillator)
3. bias (Bias)
4. bop (Balance of Power)
5. brar (AR/BR Ratio)
6. cci (Commodity Channel Index)
7. cfo (Chande Forecast Oscillator)
8. cg (Center of Gravity)
9. cmo (Chande Momentum Oscillator)
10. coppock (Coppock Curve) <- oscylator miesięczny, trzeba dostosować parametry: 14 -> 14*21=294, 11 -> 11*21=231, 10 -> 10*21=210, 
Default Inputs: length=10, fast=11, slow=14
11. dpo (Detrended Price Oscillator)
12. eri (Elder Ray Index)
13. fisher (Fisher Transform)
14. inertia (Inertia)
15. kdj (KDJ Indicator)
16. kst (KST Oscillator)
17. macd (Moving Average Convergence Divergence)
18. mom (Momentum)
19. pgo (Pretty Good Oscillator)
20. ppo (Percentage Price Oscillator)
21. psl (Psychological Line)
22. qqe (Quantitative Qualitative Estimation)
23. qstick (QStick)
24. roc (Rate of Change)
25. rsi (Relative Strength Index)
26. rsx (Relative Strength Xtra)
27. rvgi (Relative Vigor Index)
28. smi (Stochastic Momentum Index)
29. stc (Schaff Trend Cycle)
30. stoch (Stochastic Oscillator)
31. stochrsi (Stochastic RSI)
32. thermo (Thermometer)
33. trix (Trix)
34. tsi (True Strength Index)
35. uo (Ultimate Oscillator)
36. willr (Williams' %R)

Wskaźniki zmienności i kanały

1. aberration (Aberration)
2. accbands (Acceleration Bands)
3. atr (Average True Range)
4. bbands (Bollinger Bands)
5. chop (Choppiness Index)
6. cksp (Chande Kroll Stop)
7. donchian (Donchian Channels)
8. entropy (Information Entropy)
9. hilo (High-Low Channels)
10. kc (Keltner Channels)
11. massi (Mass Index)
12. natr (Normalized Average True Range)
13. rvi (Relative Volatility Index)
14. squeeze (TTM Squeeze)
15. squeeze_pro (TTM Squeeze Pro)
16. tos_stdevall (thinkorswim Standard Deviation All)
17. true_range (True Range)
18. ui (Ulcer Index)

In [149]:
# Aberration, ustawienia standardowe
# Default Inputs: length=5, atr_length=15, offset=0 <- parametry do modyfikacji
df.ta.aberration(append=True)

,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.133375,NaN,NaN,NaN
...,...,...,...,...
2025-07-28 00:00:00+00:00,1.173106,1.180681,1.165531,0.007575
2025-07-29 00:00:00+00:00,1.169141,1.176745,1.161537,0.007604
2025-07-30 00:00:00+00:00,1.163325,1.171566,1.155083,0.008242


In [150]:
# Acceleration Bands, ustawienia standardowe
# Default Inputs: length=10, c=4, mamode='sma', drift=1, offset=0 <- parametry do modyfikacji
df.ta.accbands(append=True)

,ACCBL_20,ACCBM_20,ACCBU_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.151726,1.169352,1.187469
2025-07-29 00:00:00+00:00,1.149749,1.168205,1.187262
2025-07-30 00:00:00+00:00,1.147643,1.166663,1.186866


In [151]:
# Average True Range, ustawienia standardowe
# Default Inputs: length=14, mamode='rma', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.atr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.007586
2025-07-29 00:00:00+00:00    0.007616
2025-07-30 00:00:00+00:00    0.008299
2025-07-31 00:00:00+00:00    0.008097
2025-08-01 00:00:00+00:00    0.008986
Name: ATRr_14, Length: 7152, dtype: float64

In [152]:
# Bollinger Bands, ustawienia standardowe
# Default Inputs: length=5, std=2, ddof=0, mamode='sma', talib=True, offset=0 <- parametry do modyfikacji
df.ta.bbands(append=True)

,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
timestamp,,,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,1.116562,1.135282,1.154002,3.297886,0.845029
...,...,...,...,...,...
2025-07-28 00:00:00+00:00,1.159452,1.172382,1.185312,2.205717,0.006872
2025-07-29 00:00:00+00:00,1.150370,1.167928,1.185486,3.006677,0.133555
2025-07-30 00:00:00+00:00,1.136936,1.161394,1.185852,4.211789,0.119466


In [153]:
# Choppiness Index, ustawienia standardowe
# Default Inputs: length=14, atr_length=1, ln=False, scalar=100, drift=1, offset=0 <- parametry do modyfikacji
df.ta.chop(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    57.606553
2025-07-29 00:00:00+00:00    53.837323
2025-07-30 00:00:00+00:00    44.203085
2025-07-31 00:00:00+00:00    42.808689
2025-08-01 00:00:00+00:00    43.350934
Name: CHOP_14_1_100, Length: 7152, dtype: float64

In [154]:
# Chande Kroll Stop, ustawienia standardowe
# Default Inputs: p=10, x=1, q=9, tvmode=True, offset=0 <- parametry do modyfikacji
df.ta.cksp(append=True)

,CKSPl_10_3_20,CKSPs_10_3_20
timestamp,,
2003-05-04 00:00:00+00:00,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN
...,...,...
2025-07-28 00:00:00+00:00,1.163253,1.175247
2025-07-29 00:00:00+00:00,1.162446,1.175143
2025-07-30 00:00:00+00:00,1.162446,1.166168


In [155]:
# Donchian Channels, ustawienia standardowe
# Default Inputs: lower_length=20, upper_length=20, offset=0 <- parametry do modyfikacji
df.ta.donchian(append=True)

,DCL_20_20,DCM_20_20,DCU_20_20
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.15566,1.167335,1.17901
2025-07-29 00:00:00+00:00,1.15188,1.165445,1.17901
2025-07-30 00:00:00+00:00,1.14008,1.159475,1.17887


In [156]:
# Information Entropy, ustawienia standardowe
# Default Inputs: length=10, base=2, offset=0 <- parametry do modyfikacji
df.ta.entropy(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    3.325034
2025-07-29 00:00:00+00:00    3.323890
2025-07-30 00:00:00+00:00    3.320776
2025-07-31 00:00:00+00:00    3.317604
2025-08-01 00:00:00+00:00    3.316120
Name: ENTP_10, Length: 7152, dtype: float64

In [157]:
# High-Low Channels, ustawienia standardowe
# Default Inputs: high_length=13, low_length=21, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.hilo(append=True)

,HILO_13_21,HILOl_13_21,HILOs_13_21
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.172289,NaN,1.172289
2025-07-29 00:00:00+00:00,1.171528,NaN,1.171528
2025-07-30 00:00:00+00:00,1.170605,NaN,1.170605


In [158]:
# Keltner Channels, ustawienia standardowe
# Default Inputs: length=20, scalar=2, mamode='ema', offset=0 <- parametry do modyfikacji
df.ta.kc(append=True)

,KCLe_20_2,KCBe_20_2,KCUe_20_2
timestamp,,,
2003-05-04 00:00:00+00:00,NaN,NaN,NaN
2003-05-05 00:00:00+00:00,NaN,NaN,NaN
2003-05-06 00:00:00+00:00,NaN,NaN,NaN
2003-05-07 00:00:00+00:00,NaN,NaN,NaN
2003-05-08 00:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2025-07-28 00:00:00+00:00,1.153341,1.168734,1.184128
2025-07-29 00:00:00+00:00,1.151979,1.167432,1.182885
2025-07-30 00:00:00+00:00,1.147832,1.165084,1.182336


In [159]:
# Mass Index, ustawienia standardowe
# Default Inputs: fast=9, slow=25, offset=0 <- parametry do modyfikacji
df.ta.massi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    24.544247
2025-07-29 00:00:00+00:00    24.753080
2025-07-30 00:00:00+00:00    25.100503
2025-07-31 00:00:00+00:00    25.298507
2025-08-01 00:00:00+00:00    25.605867
Name: MASSI_9_25, Length: 7152, dtype: float64

In [160]:
# Normalized Average True Range, ustawienia standardowe
# Default Inputs: length=20, scalar=100, mamode='ema', talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.natr(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    0.686464
2025-07-29 00:00:00+00:00    0.689752
2025-07-30 00:00:00+00:00    0.804539
2025-07-31 00:00:00+00:00    0.761611
2025-08-01 00:00:00+00:00    0.887310
Name: NATR_14, Length: 7152, dtype: float64

In [161]:
# Relative Volatility Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, refined=False, thirds=False, mamode='ema', drift=1, offset=0 <- parametry do modyfikacji
df.ta.rvi(append=True)

timestamp
2003-05-04 00:00:00+00:00          NaN
2003-05-05 00:00:00+00:00          NaN
2003-05-06 00:00:00+00:00          NaN
2003-05-07 00:00:00+00:00          NaN
2003-05-08 00:00:00+00:00          NaN
                               ...    
2025-07-28 00:00:00+00:00    49.893494
2025-07-29 00:00:00+00:00    41.993566
2025-07-30 00:00:00+00:00    33.707001
2025-07-31 00:00:00+00:00    26.550435
2025-08-01 00:00:00+00:00    41.104084
Name: RVI_14, Length: 7152, dtype: float64

In [162]:
# TTM Squeeze, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar=1.5, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze(append=True)

,SQZ_20_2.0_20_1.5,SQZ_ON,SQZ_OFF,SQZ_NO
timestamp,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,1
...,...,...,...,...
2025-07-28 00:00:00+00:00,0.002693,0,1,0
2025-07-29 00:00:00+00:00,0.001512,0,1,0
2025-07-30 00:00:00+00:00,-0.002538,0,1,0


In [163]:
# TTM Squeeze Pro, ustawienia standardowe
# Default Inputs: bb_length=20, bb_std=2, kc_length=20, kc_scalar_wide=2, kc_scalar_normal=1.5, kc_scalar_narrow=1, mom_length=12, mom_smooth=6, use_tr=True, mamode='sma', offset=0 <- parametry do modyfikacji
df.ta.squeeze_pro(append=True)

,SQZPRO_20_2.0_20_2_1.5_1,SQZPRO_ON_WIDE,SQZPRO_ON_NORMAL,SQZPRO_ON_NARROW,SQZPRO_OFF,SQZPRO_NO
timestamp,,,,,,
2003-05-04 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-05 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-06 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-07 00:00:00+00:00,NaN,0,0,0,0,1
2003-05-08 00:00:00+00:00,NaN,0,0,0,0,1
...,...,...,...,...,...,...
2025-07-28 00:00:00+00:00,0.002693,1,0,0,0,0
2025-07-29 00:00:00+00:00,0.001512,1,0,0,0,0
2025-07-30 00:00:00+00:00,-0.002538,0,0,0,1,0


In [164]:
# Think or Swim Standard Deviation All, ustawienia standardowe
# Default Inputs: length=None (means All), stds=[1,2,3], ddof=1, offset=0 <- parametry do modyfikacji
df.ta.tos_stdevall(append=True)

,TOS_STDEVALL_LR,TOS_STDEVALL_L_1,TOS_STDEVALL_U_1,TOS_STDEVALL_L_2,TOS_STDEVALL_U_2,TOS_STDEVALL_L_3,TOS_STDEVALL_U_3
timestamp,,,,,,,
2003-05-04 00:00:00+00:00,1.369040,1.241146,1.496935,1.113252,1.624829,0.985357,1.752723
2003-05-05 00:00:00+00:00,1.369001,1.241106,1.496895,1.113212,1.624789,0.985318,1.752684
2003-05-06 00:00:00+00:00,1.368961,1.241067,1.496856,1.113173,1.624750,0.985278,1.752644
2003-05-07 00:00:00+00:00,1.368922,1.241027,1.496816,1.113133,1.624710,0.985239,1.752605
2003-05-08 00:00:00+00:00,1.368882,1.240988,1.496777,1.113094,1.624671,0.985199,1.752565
...,...,...,...,...,...,...,...
2025-07-28 00:00:00+00:00,1.086748,0.958854,1.214642,0.830959,1.342537,0.703065,1.470431
2025-07-29 00:00:00+00:00,1.086708,0.958814,1.214603,0.830920,1.342497,0.703026,1.470391
2025-07-30 00:00:00+00:00,1.086669,0.958775,1.214563,0.830880,1.342458,0.702986,1.470352


In [165]:
# True Range, ustawienia standardowe
# Default Inputs: talib=True, drift=1, offset=0 <- parametry do modyfikacji
df.ta.true_range(append=True)

timestamp
2003-05-04 00:00:00+00:00        NaN
2003-05-05 00:00:00+00:00    0.00995
2003-05-06 00:00:00+00:00    0.01783
2003-05-07 00:00:00+00:00    0.01058
2003-05-08 00:00:00+00:00    0.02071
                              ...   
2025-07-28 00:00:00+00:00    0.01850
2025-07-29 00:00:00+00:00    0.00801
2025-07-30 00:00:00+00:00    0.01717
2025-07-31 00:00:00+00:00    0.00548
2025-08-01 00:00:00+00:00    0.02054
Name: TRUERANGE_1, Length: 7152, dtype: float64

In [166]:
# Ulcer Index, ustawienia standardowe
# Default Inputs: length=14, scalar=100, offset=0 <- parametry do modyfikacji
df.ta.ui(append=True)

timestamp
2003-05-04 00:00:00+00:00         NaN
2003-05-05 00:00:00+00:00         NaN
2003-05-06 00:00:00+00:00         NaN
2003-05-07 00:00:00+00:00         NaN
2003-05-08 00:00:00+00:00         NaN
                               ...   
2025-07-28 00:00:00+00:00    1.074635
2025-07-29 00:00:00+00:00    1.154609
2025-07-30 00:00:00+00:00    1.359337
2025-07-31 00:00:00+00:00    1.511260
2025-08-01 00:00:00+00:00    1.524112
Name: UI_14, Length: 7152, dtype: float64

In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7152 entries, 2003-05-04 00:00:00+00:00 to 2025-08-01 00:00:00+00:00
Data columns (total 85 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   open                      7152 non-null   float64
 1   high                      7152 non-null   float64
 2   low                       7152 non-null   float64
 3   close                     7152 non-null   float64
 4   volume                    7152 non-null   float64
 5   ADX_14                    7125 non-null   float64
 6   DMP_14                    7138 non-null   float64
 7   DMN_14                    7138 non-null   float64
 8   AMATe_LR_8_21_2           7152 non-null   int32  
 9   AMATe_SR_8_21_2           7152 non-null   int32  
 10  AROOND_14                 7138 non-null   float64
 11  AROONU_14                 7138 non-null   float64
 12  AROONOSC_14               7138 non-null   float64
 13  CTI_12         

Wskaźniki wolumenu

1. ad (Accumulation/Distribution Line)
2. adosc (Chaikin Oscillator)
3. aobv (Adaptive On-Balance Volume)
4. cmf (Chaikin Money Flow)
5. efi (Elder's Force Index)
6. eom (Ease of Movement)
7. kvo (Klinger Volume Oscillator)
8. mfi (Money Flow Index)
9. nvi (Negative Volume Index)
10. obv (On-Balance Volume)
11. pvi (Positive Volume Index)
12. pvo (Percentage Volume Oscillator)
13. pvol (Price-Volume)
14. pvr (Price-Volume Rank)
15. pvt (Price and Volume Trend)
16. vp (Volume Profile)
17. vwap (Volume-Weighted Average Price)
18. vwma (Volume-Weighted Moving Average)

Funkcje rozpoznające formacje świecowe

cdl_pattern (Candlestick Pattern)

cdl_z (Candlestick Z-Score)

Średnie Kroczące, filtry i funkcje wygładzające średnie

alma (Arnaud Legoux Moving Average)

dema (Double Exponential Moving Average)

ema (Exponential Moving Average)

fwma (Fibonacci's Weighted Moving Average)

hma (Hull Moving Average)

hwma (Holt-Winters Moving Average)

jma (Jurik Moving Average)

kama (Kaufman's Adaptive Moving Average)

mcgd (McGinley Dynamic)

pwma (Pascal's Weighted Moving Average)

rma (Running Moving Average)

sinwma (Sine Weighted Moving Average)

sma (Simple Moving Average)

ssf (Ehlers Super Smoother Filter)

swma (Symmetric Weighted Moving Average)

t3 (T3 Moving Average)

tema (Triple Exponential Moving Average)

trima (Triangular Moving Average)

vidya (Variable Index Dynamic Average)

wma (Weighted Moving Average)

zlma (Zero Lag Moving Average)

Miary statystyczne i pomocnicze

decay (Decay)

hwc / wcp (Heavily Weighted Close / Weighted Close Price)

hl2 (High-Low 2)

hlc3 (High-Low-Close 3)

kurtosis (Kurtoza)

log_return (Logarithmic Return)

mad (Mean Absolute Deviation)

median (Mediana)

midpoint (Midpoint)

midprice (Midprice)

ohlc4 (Open-High-Low-Close 4)

pdist (Price Distance)

percent_return (Percent Return)

quantile (Kwantyl)

skew (Skośność)

stdev (Standard Deviation)

variance (Wariancja)

zscore (Z-Score)